## “모르겠다”, “수업을 안들어서 모른다”, “기억 안남” 등의 응답을 NaN 처리하고 결측값 제거하기

1. **문장 유사도 기반 NaN 처리**  
   - `SentenceTransformer`를 이용해 기준 문장(“모르겠다”, “들어본 적 없음”, “기억 안난다” 등)을 먼저 임베딩하여 저장  
   - 각 리뷰 문장에 대해 동일한 임베딩 모델로 벡터화하고, 기준 임베딩들과의 코사인 유사도를 계산  
   - 유사도가 임계치(예: 0.85) 이상인 경우 `np.nan`으로 반환하여 “모름” 응답을 결측값으로 표시  

2. **결측값 제거**  
   - Pandas의 `dropna()` 또는 `DataFrame.dropna(axis=0, how='any')` 메서드를 사용해 NaN이 된 행(응답)들을 제거  
   - 이렇게 하면 실제 수업을 들은 경험에 기반한 리뷰만 남겨, 이후 분석의 왜곡을 막습니다  

In [ ]:
import pandas as pd

In [ ]:
# 엑셀 파일 경로
excel_path = '/content/sample_data/professor_review.xlsx'

# 엑셀 파일 읽기
df = pd.read_excel(excel_path)

# CSV 파일로 저장 (원하는 경로와 이름으로 변경 가능)
csv_path = '/content/sample_data/data.csv'
df.to_csv(csv_path, index=False)

print(f'CSV 파일로 저장 완료: {csv_path}')

CSV 파일로 저장 완료: /content/sample_data/data.csv


/usr/local/lib/python3.11/dist-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [ ]:
df = pd.read_csv('/content/sample_data/data.csv', header=0)

In [ ]:
df.head()

,응답일시,참여자,김유섭 교수님,양은샘 교수님,신미영 교수님,김선정 교수님,이정근 교수님,김은주 교수님
0,2025-05-13 18:51:32,1,"친근하다,적극적이다,좋다","친근하다,적극적이다,좋다","친근하다,적극적이다,좋다","친근하다,적극적이다,좋다","친근하다,적극적이다,좋다","친근하다,적극적이다,좋다"
1,2025-05-13 18:56:29,2,NaN,NaN,NaN,매 수업마다 pdf파일을 올려주시며 그 pdf에 나온 대로 따라만 하면 어느정도 수...,NaN,NaN
2,2025-05-13 19:33:43,3,교수님의 말투가 친근하고 또박또박 말씀하셔서 듣기가 좋습니다. 강의 자료만 봐도 이...,과제가 좀 많은게 단점이지만 그만큼 배워 갈수있는것도 많은 교수님이신것같습니다. 수...,강의 자료가 좋아서 교수님중에서 가장 배워갈께 많은 교수님인것같습니다. 다만 신미영...,NaN,이정근 교수님은 굉장히 친절하십니다. 강의자료가 영어로 되어있고 보기 힘들었지만 수...,NaN
3,2025-05-13 20:01:56,4,말투가 느리시지만 상냥한 어투로 말해주신다. 강의 자료는 깔끔하고 이해가 쉽게 되어...,"학생들에게 친근하게 대해주시고, 수업 시에 딕션이 좋으시다. 강의 자료는 이해하기 ...",교수님 말투가 전달력이 좋으신 편은 아니시다. 강의 자료는 이해하기 쉽게 깔끔하게 ...,"수업 속도가 빠른 편이고, 이해하기 쉽게 설명해 주신다. 과제는 매주 있긴 하지만 ...","말이 느리시지만, 학생들에게 친절하게 말해주신다. 수업은 이해가 쉽도록 학생들 입장...",딕션이 좋으시고 말이 굉장히 빠르시다. 말이 빠르셔서 그냥 귀에서 스쳐갈 때도 있지...
4,2025-05-13 20:05:14,5,NaN,수업 내용이 많지만 그만큼 알차다. 과제는 짧으면 1시간 오래걸리면 2~3시간까지 ...,NaN,NaN,NaN,"말투가 굉장이 빠르시고, 강의는 쉬운 편이다. 과제량도 매주 있지만 1시간이면 끝날..."


In [ ]:
print(df.columns)

Index(['응답일시', '참여자', '김유섭 교수님', '양은샘 교수님', '신미영 교수님', '김선정 교수님', '이정근 교수님',
       '김은주 교수님'],
      dtype='object')


In [ ]:
# 교수님별 컬럼 리스트 (예: 첫 2개 컬럼은 시간, 참여자 정보라고 가정)
professor_cols = df.columns[2:]

# 교수님별 리뷰 개별 리스트 추출 예시
for prof in professor_cols:
    print(f'[{prof}] 리뷰 개수:', df[prof].dropna().shape[0])

[김유섭 교수님] 리뷰 개수: 43
[양은샘 교수님] 리뷰 개수: 34
[신미영 교수님] 리뷰 개수: 31
[김선정 교수님] 리뷰 개수: 29
[이정근 교수님] 리뷰 개수: 33
[김은주 교수님] 리뷰 개수: 33


In [ ]:
for prof in professor_cols:
    print(f'[{prof}] NaN 개수:',df[prof].isna().sum())

[김유섭 교수님] NaN 개수: 7
[양은샘 교수님] NaN 개수: 16
[신미영 교수님] NaN 개수: 19
[김선정 교수님] NaN 개수: 21
[이정근 교수님] NaN 개수: 17
[김은주 교수님] NaN 개수: 17


In [ ]:
pip install sentence-transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 102.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 85.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 46.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 85.3 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitli

In [ ]:
from sentence_transformers import SentenceTransformer, util
import numpy as np

# 1) 임베딩 모델 로드 (가볍고 빠른 모델 추천)
model = SentenceTransformer('all-MiniLM-L6-v2')

# 2) 기준 문장 리스트 (수업 안 들음/모름 의미)
reference_texts = [
    "수업을 들은 적이 없다",
    "들어본 적 없음",
    "모른다",
    "수강한 적 없음",
    "들어본적이 없어요",
    "머름",
    "잘 모릅니다",
    "기억이 안난다",
]

# 기준 문장 임베딩
ref_embeddings = model.encode(reference_texts, convert_to_tensor=True)

def to_nan_if_similar(text, threshold=0.85):
    if not isinstance(text, str):
        return text
    emb = model.encode(text, convert_to_tensor=True)
    cosine_scores = util.pytorch_cos_sim(emb, ref_embeddings)
    max_score = cosine_scores.max().item()
    if max_score > threshold:
        print(f"NaN 처리됨: '{text}', 유사도: {max_score}")
        return np.nan
    return text

# professor_cols는 교수님별 컬럼 리스트
for col in professor_cols:
    df[col] = df[col].apply(to_nan_if_similar)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

NaN 처리됨: '잘 기억이 안난다.', 유사도: 0.944824755191803
NaN 처리됨: '모름', 유사도: 0.9165349006652832
NaN 처리됨: '수업을 들은 적이 없다.', 유사도: 0.9800688028335571
NaN 처리됨: '잘 모릅니다', 유사도: 1.0
NaN 처리됨: '오래돼서 기억이 안난다', 유사도: 0.9895537495613098
NaN 처리됨: '모른다', 유사도: 1.000000238418579
NaN 처리됨: '들은 적이 없습니다', 유사도: 0.927575945854187
NaN 처리됨: '수업 들어본적 없음', 유사도: 0.9380893707275391
NaN 처리됨: '적극적이다', 유사도: 0.8764221668243408
NaN 처리됨: '수업을 들은 적이 없다.', 유사도: 0.9800688028335571
NaN 처리됨: '잘 모릅니다', 유사도: 1.0
NaN 처리됨: '오래돼서 기억이 안남.', 유사도: 0.8729536533355713
NaN 처리됨: '위 교수님의 수업을 들은 적 없음', 유사도: 0.8588711619377136
NaN 처리됨: '모른다', 유사도: 1.000000238418579
NaN 처리됨: '들은 적이 없습니다', 유사도: 0.927575945854187
NaN 처리됨: '수업을 들어본적이 없어서 잘 모르겠습니다.', 유사도: 0.9343311786651611
NaN 처리됨: '수업을 듣지 않아서 잘 모르겠다.', 유사도: 0.8624138236045837
NaN 처리됨: '머름', 유사도: 1.0
NaN 처리됨: '수업을 들은 적이 없다.', 유사도: 0.9800688028335571
NaN 처리됨: '잘 모릅니다', 유사도: 1.0
NaN 처리됨: '들은 적 없음.', 유사도: 0.9235415458679199
NaN 처리됨: '모른다', 유사도: 1.000000238418579
NaN 처리됨: '들은 적이 없습니다', 유사도: 0.927575945854187


In [ ]:
professor_cols = ['김유섭 교수님', '양은샘 교수님', '신미영 교수님', '김선정 교수님', '이정근 교수님', '김은주 교수님']

# 각 컬럼에 함수 적용해서 NaN 처리
for col in professor_cols:
    df[col] = df[col].apply(to_nan_if_similar)


In [ ]:
for prof in professor_cols:
    print(f'[{prof}] NaN 개수:',df[prof].isna().sum())

[김유섭 교수님] NaN 개수: 7
[양은샘 교수님] NaN 개수: 24
[신미영 교수님] NaN 개수: 27
[김선정 교수님] NaN 개수: 29
[이정근 교수님] NaN 개수: 23
[김은주 교수님] NaN 개수: 23


In [ ]:
import pandas as pd

all_reviews = []
for col in professor_cols:
    temp = (
        df[col]
        .dropna()
        .to_frame(name="review")
        .assign(professor=col)
        [['professor', 'review']]
    )
    all_reviews.append(temp)

combined_df = pd.concat(all_reviews, ignore_index=True)

# CSV로 저장 (왼쪽: professor, 오른쪽: review)
combined_df.to_csv("all_professors_reviews.csv",
                   index=False,
                   encoding="utf-8-sig")


In [ ]:
combined_df = pd.read_csv("all_professors_reviews.csv", encoding="utf-8-sig")

print(combined_df.head(5).to_string(index=False))

professor                                                                                                    review
  김유섭 교수님                                                                                             친근하다,적극적이다,좋다
  김유섭 교수님                                               교수님의 말투가 친근하고 또박또박 말씀하셔서 듣기가 좋습니다. 강의 자료만 봐도 이해하기 좋은 수업입니다.
  김유섭 교수님 말투가 느리시지만 상냥한 어투로 말해주신다. 강의 자료는 깔끔하고 이해가 쉽게 되어있었고, 수업도 이해하기 쉽게 알려주신다.\n시험은 어렵게 내시 지도 않고, 쉽지도 않은 평균인 것 같다.
  김유섭 교수님                                                              교수님 인상과 말투가 친근하다\n수업이해도 보통이다\n학생 수업 참여 유도 없다
  김유섭 교수님                                      교수님 설명을 쉽게 잘해주신다. 시험 난이도는 적당한 편이다. 적극적으로 수업 하신다. 과제랑 수업의 연관성이 높은편이다.


**문장 유사도 기반 NaN 처리 결측값 제거하기 전 후 결과 비교**

In [ ]:
#결측값 제거 전
professor_cols = df.columns[2:]

# 교수님별 리뷰 개별 리스트 추출 예시
for prof in professor_cols:
    print(f'[{prof}] 리뷰 개수:', df[prof].dropna().shape[0])

[김유섭 교수님] 리뷰 개수: 43
[양은샘 교수님] 리뷰 개수: 26
[신미영 교수님] 리뷰 개수: 23
[김선정 교수님] 리뷰 개수: 21
[이정근 교수님] 리뷰 개수: 27
[김은주 교수님] 리뷰 개수: 27


In [ ]:
#결측값 제거 후
#'모른다' '수업을 안들어서 모르겠다'를  NaN처리

print("=== 교수님별 리뷰 개수 (value_counts) ===")
print(combined_df['professor'].value_counts().to_string())

=== 교수님별 리뷰 개수 (value_counts) ===
professor
김유섭 교수님    43
김은주 교수님    27
이정근 교수님    27
양은샘 교수님    26
신미영 교수님    23
김선정 교수님    21


In [ ]:
# 직접 수작업으로 모두 처리
reviews = pd.read_csv("reviews.csv", encoding="utf-8-sig")

print("=== 교수님별 리뷰 개수 (value_counts) ===")
print(reviews['professor'].value_counts().to_string())

=== 교수님별 리뷰 개수 (value_counts) ===
professor
김유섭 교수님    43
김은주 교수님    27
이정근 교수님    26
양은샘 교수님    24
신미영 교수님    23
김선정 교수님    20
